# Azure Open AI - surveys - part 2 Search

In [1]:
# %pip install azure-search-documents --pre --upgrade

In [2]:
# %pip install youtube-search-python

In [3]:
import json
import openai
import os
import pytz
import re
import requests
import sys
import time

from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.models import Vector

from datetime import datetime
from dotenv import load_dotenv
from IPython.display import IFrame, display
from io import BytesIO

In [4]:
sys.version

'3.10.10 (main, Mar 21 2023, 18:45:11) [GCC 11.2.0]'

In [5]:
local_tz = pytz.timezone(requests.get("https://ipinfo.io").json()["timezone"])
print("Local time:", datetime.now(local_tz).strftime("%d-%b-%Y %H:%M:%S"))

Local time: 08-Sep-2023 15:04:52


In [6]:
print("Open AI version:", openai.__version__)

Open AI version: 0.28.0


In [7]:
load_dotenv("azure.env")

openai.api_type: str = "azure"
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_version = os.getenv("OPENAI_API_VERSION")

acs_endpoint = os.getenv("AZURE_COGNITIVE_SEARCH_ENDPOINT")
acs_key = os.getenv("AZURE_COGNITIVE_SEARCH_API_KEY")

In [8]:
# Azure Open AI embeddings model to use
embeddings_engine = "text-embedding-ada-002"

- Vector search is in public preview
- Model name: text-embedding-ada-002
- Model version: 2
- API version: 2023-05-15

In [9]:
# Azure Cognitive Search index name to create
index_name = "surveys"

## 1. Functions

In [10]:
def acs_delete_index(index_name):
    """
    Deleting an Azure Cognitive Search index
    Input: an Azure Cognitive Search index name
    """
    start = time.time()
    search_client = SearchIndexClient(
        endpoint=acs_endpoint, credential=AzureKeyCredential(acs_key)
    )
    
    try:
        print("Deleting the Azure Cognitive Search index:", index_name)
        search_client.delete_index(index_name)
        print("Done. Elapsed time:", round(time.time() - start, 2), "secs")
    except:
        print("Cannot delete index. Check the index name.")


In [11]:
def acs_index_stats(index_name):
    """
    Get statistics about Azure Cognitive Search index
    Input: an Azure Cognitive Search index name
    """
    url = (
        acs_endpoint
        + "/indexes/"
        + index_name
        + "/stats?api-version=2021-04-30-Preview"
    )
    headers = {
        "Content-Type": "application/json",
        "api-key": acs_key,
    }
    response = requests.get(url, headers=headers)
    print("Azure Cognitive Search index status for:", index_name, "\n")

    if response.status_code == 200:
        res = response.json()
        print(json.dumps(res, indent=2))
        document_count = res["documentCount"]
        storage_size = res["storageSize"]

    else:
        print("Request failed with status code:", response.status_code)

    return document_count, storage_size

In [12]:
def acs_index_status(index_name):
    """
    Azure Cognitive Search index status
    Input: an Azure Cognitive Search index name
    """
    print("Azure Cognitive Search Index:", index_name, "\n")

    headers = {"Content-Type": "application/json", "api-key": acs_key}
    params = {"api-version": "2021-04-30-Preview"}
    index_status = requests.get(
        acs_endpoint + "/indexes/" + index_name, headers=headers, params=params
    )

    try:
        print(json.dumps((index_status.json()), indent=5))
    except:
        print("Request failed with status code:", response.status_code)

In [13]:
def openai_text_embeddings(text):
    """
    Generating embeddings from text using Azure Open AI
    Input: a text
    Output: a vector embedding made by Azure Open AI
    """
    embeddings = openai.Embedding.create(
        input=text,
        deployment_id=embeddings_engine,
    )
    embeddings = embeddings["data"][0]["embedding"]

    return embeddings

In [14]:
acs_index_status(index_name)

Azure Cognitive Search Index: surveys 

{
     "@odata.context": "https://azurecogsearcheastussr.search.windows.net/$metadata#indexes/$entity",
     "@odata.etag": "\"0x8DBAFC1E451A7AF\"",
     "name": "surveys",
     "defaultScoringProfile": null,
     "fields": [
          {
               "name": "id",
               "type": "Edm.String",
               "searchable": false,
               "filterable": true,
               "retrievable": true,
               "sortable": true,
               "facetable": true,
               "key": true,
               "indexAnalyzer": null,
               "searchAnalyzer": null,
               "analyzer": null,
               "normalizer": null,
               "synonymMaps": []
          },
          {
               "name": "company",
               "type": "Edm.String",
               "searchable": true,
               "filterable": true,
               "retrievable": true,
               "sortable": false,
               "facetable": false,
     

In [15]:
document_count, storage_size = acs_index_stats(index_name)

Azure Cognitive Search index status for: surveys 

{
  "@odata.context": "https://azurecogsearcheastussr.search.windows.net/$metadata#Microsoft.Azure.Search.V2021_04_30_Preview.IndexStatistics",
  "documentCount": 1060,
  "storageSize": 27980054
}


In [16]:
print("Number of documents in the index =", f"{document_count:,}")
print("Size of the index =", round(storage_size / (1024 * 1024), 2), "MB")

Number of documents in the index = 1,060
Size of the index = 26.68 MB


In [17]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

## 3. Pure Vector Search

In [18]:
def acs_vectorsearch_query(query, topn=5):
    """
    Find comments from Azure Cognitive Search 
    Pure Vector Search
    Input: a query (string)
    Output: results (list)
    """
    start = time.time()
    search_client = SearchClient(
        acs_endpoint, index_name, credential=AzureKeyCredential(acs_key)
    )

    vector = Vector(
        value=openai_text_embeddings(query), k=topn, fields="embed_text"
    )

    results = search_client.search(
        search_text=None,
        vectors=[vector],
        select=["id", "company", "comments", "month", "year", "text"],
    )

    results_list = []

    idx = 1
    for result in results:
        print("\033[1;31;34m")
        print(f"{idx} ID: {result['id']}")
        print("company:", result["company"])
        print("\033[1;31;32m")
        print(f"comments: {result['comments']}")
        print("\033[0m")
        print(f"month: {result['month']}")
        print(f"year: {result['year']}")
        print("\033[1;31;35m")
        print(f"Score = {result['@search.score']}")
        idx += 1

        results_list.append(result["id"])

    elapsed = time.time() - start
    print("\033[0m")
    print(
        "\nElapsed time: "
        + time.strftime(
            "%H:%M:%S.{}".format(str(elapsed % 1)[2:])[:15], time.gmtime(elapsed)
        )
    )

    return results_list

**Multi-lingual search**. Use a multi-lingual embeddings model to represent your document in multiple languages in a single vector space to find documents regardless of the language they are in.


In [19]:
acs_vectorsearch_query("soins dentaires")


1 ID: 190
company: MonAssurance

comments: un très bon suivi même si quand on veut de très grosse ganranties il faut payer unpeu cher mais j ai changé quand je n ai plus eu besoins de dentaire et je paye 116€ pour moi ma femme et mes 3 enfants pour des garanties très correctes

month: 5
year: 2014

Score = 0.8591364

2 ID: 104
company: Réassurez-moi

comments: Votre cabinet m'a permis de pouvoir bénéficier du contrat de complémentaire santé le plus adéquat en fonction de mes besoins en matière de soins dentaires. Les remboursements des soins se font dans des délais rapides. De plus, je garde le même interlocuteur (merci Charles !) pour toutes les questions que j'ai à poser, et ce dernier est toujours disponible et très aimable :-)

month: 8
year: 2019

Score = 0.858289

3 ID: 105
company: Réassurez-moi

comments: Nous avons cherché pendant quelque temps une mutuelle santé qui prendrait en charge mon traitement d'orthodontie de façon plus complète. Nous l'avons trouvé avec l'aide de ré

['190', '104', '105', '817', '172']

In [20]:
acs_vectorsearch_query("conseillère 'Santiane'")


1 ID: 638
company: Santiane

comments: Chez Santiane depuis 2014, j'ai été recontacté par le service client aujourd'hui. La conseillère clients a été très aimable et patiente et les conseils prodigués, judicieux. Je suis très satisfait. Cantissimo.

month: 6
year: 2018

Score = 0.8839418

2 ID: 450
company: Santiane

comments: Je suis cliente chez santiane depuis 6 ans et j'ai toujours eu un très service agréable . Ils sont à l'écoute et très aimables au téléphone. Sabrina est très sympathique avec moi malgré mon incompréhension.

month: 7
year: 2019

Score = 0.8784466

3 ID: 635
company: Santiane

comments: Travailleur indépendant, je suis pleinement satisfait d'avoir choisi Santiane il y a maintenant 6 ans._x000D_
_x000D_
Une petite évaluation de mes besoins, et en quelques minutes, un devis pour une offre adaptée, quasiment aucune démarche à effectuer._x000D_
_x000D_
Santiane se charge de me contacter régulièrement afin de me proposer des offres suivant l'évolution de mes besoins e

['638', '450', '635', '537', '732']

## 4.3 Pure vector search with filter
We can specify a filter on the vector search

In [21]:
myfilter = "year eq '2019' and company eq 'Santiane'"
myfilter

"year eq '2019' and company eq 'Santiane'"

In [22]:
start = time.time()

query = "orthodontie "

search_client = SearchClient(
    acs_endpoint, index_name, credential=AzureKeyCredential(acs_key)
)

vector = Vector(value=openai_text_embeddings(query), k=5, fields="embed_text")

results = search_client.search(
    search_text=None,
    vectors=[vector],
    filter=myfilter,
    select=["id", "company", "comments", "month", "year"],
)

idx = 1
results_list= []
for result in results:
    print("\033[1;31;34m")
    print(f"{idx} ID: {result['id']}")
    print("company:", result["company"])
    print("\033[1;31;32m")
    print(f"Comments: {result['comments']}")
    print("\033[0m")
    print(f"Month: {result['month']}")
    print(f"Year: {result['year']}")
    print("\033[1;31;35m")
    print(f"Score = {result['@search.score']}")
    idx += 1

    results_list.append(result["id"])


elapsed = time.time() - start
print("\033[0m")
print(
    "\nElapsed time: "
    + time.strftime(
        "%H:%M:%S.{}".format(str(elapsed % 1)[2:])[:15], time.gmtime(elapsed)
    )
)


1 ID: 577
company: Santiane

Comments: Lors des discussions préalables à aucun moment le service client m'a informé d'un délai de carence de 3 mois sur l'optique et le dentaire ni m'a informé que j'étais compatible avec la suppression de ce délai de carence (même groupe assureur Malakoff Médéric et même niveau de remboursement sur l'optique. J'invite donc les internautes à éviter Santiane pour son manque de professionnalisme complet.

Month: 2
Year: 2019

Score = 0.8313881

2 ID: 469
company: Santiane

Comments: Ma de mande et ma conversation avec Sabrina se sont très bien passés aujourd'hui, ce qui n'a pas toujours été le cas avec d'autres conseillers. J'ai attendu 15 jours avant d'avoir une réponse sur un devis de soins dentaires ce qui me parait inaxeptable, surtout quand on vous donne comme réponse ...soyez patient. J'espère qu'a l'avenir tout sera réglé plus rapidement et surtout avec un peu plus d'enpatie.

Month: 6
Year: 2019

Score = 0.82607275

3 ID: 373
company: Santiane

Co

## 4.4 Hybrid search

**Hybrid search consists of keyword queries and vector queries** in a single search request.

Vector search is implemented at the field level, which means you can build queries that include vector fields and searchable text fields. The queries execute in parallel and the results are merged into a single response. Optionally, add semantic search (preview) for even more accuracy with L2 reranking using the same language models that power Bing.

In [23]:
start = time.time()

query = "Néoliane Santé"

search_client = SearchClient(acs_endpoint, index_name, AzureKeyCredential(acs_key))

vector = Vector(value=openai_text_embeddings(query), k=10, fields="embed_text")

results = search_client.search(
    search_text=query,
    vectors=[vector],
    select=["id", "company", "comments", "month", "year"],
    top=10,
)

idx = 1
results_list= []
for result in results:
    print("\033[1;31;34m")
    print(f"{idx} ID: {result['id']}")
    print("company:", result["company"])
    print("\033[1;31;32m")
    print(f"Comments: {result['comments']}")
    print("\033[0m")
    print(f"Month: {result['month']}")
    print(f"Year: {result['year']}")
    print("\033[1;31;35m")
    print(f"Score = {result['@search.score']}")
    idx += 1

    results_list.append(result["id"])

elapsed = time.time() - start
print("\033[0m")
print(
    "\nElapsed time: "
    + time.strftime(
        "%H:%M:%S.{}".format(str(elapsed % 1)[2:])[:15], time.gmtime(elapsed)
    )
)


1 ID: 988
company: Néoliane Santé 

Comments: Je suis cliente chez Néoliane depuis 5 ans, aucun soucis à déclarer, remboursements rapides et prestations en adéquation avec le contrat souscris.

Month: 4
Year: 2019

Score = 0.0317540317773819

2 ID: 1088
company: Néoliane Santé 

Comments: Bonjour,_x000D_
Je m'occupe de ma mère._x000D_
Début septembre 2018, nous avons reçu une mise en demeure de l'hôpital Lavéran pour des frais de janvier. Durant 6 mois, l'hôpital n'a pas à obtenu de réponse de Néoliane._x000D_
Nous avons contacté Néolianee par la messagerie de l'espace adhérent (comme nous l'avait conseillé un opérateur de Néoliane lors de l'adhésion). _x000D_
Mais sans réponse de Néoliane, nous avons du réglé les factures._x000D_
Nous avons renvoyé messages, à ce jour sans réponse...

Month: 12
Year: 2018

Score = 0.03057890012860298

3 ID: 1063
company: Néoliane Santé 

Comments: Encore un appel ce matin d'une personne se présentant comme NEOLIANE : UNE ANOMALIE A ÉTÉ DÉTECTÉE DANS 

## 4.5 Hybrid search with a filter

In [24]:
myfilter = "year eq '2019'"

In [25]:
start = time.time()

query = "soins dentaires"

search_client = SearchClient(acs_endpoint, index_name, AzureKeyCredential(acs_key))

vector = Vector(value=openai_text_embeddings(query), k=10, fields="embed_text")

results = search_client.search(
    search_text=query,
    vectors=[vector],
    filter=myfilter,
    select=["id", "company", "comments", "month", "year"],
    top=10,
)

idx = 1
results_list= []
for result in results:
    print("\033[1;31;34m")
    print(f"{idx} ID: {result['id']}")
    print("company:", result["company"])
    print("\033[1;31;32m")
    print(f"Comments: {result['comments']}")
    print("\033[0m")
    print(f"Month: {result['month']}")
    print(f"Year: {result['year']}")
    print("\033[1;31;35m")
    print(f"Score = {result['@search.score']}")
    idx += 1

    results_list.append(result["id"])

elapsed = time.time() - start
print("\033[0m")
print(
    "\nElapsed time: "
    + time.strftime(
        "%H:%M:%S.{}".format(str(elapsed % 1)[2:])[:15], time.gmtime(elapsed)
    )
)


1 ID: 104
company: Réassurez-moi

Comments: Votre cabinet m'a permis de pouvoir bénéficier du contrat de complémentaire santé le plus adéquat en fonction de mes besoins en matière de soins dentaires. Les remboursements des soins se font dans des délais rapides. De plus, je garde le même interlocuteur (merci Charles !) pour toutes les questions que j'ai à poser, et ce dernier est toujours disponible et très aimable :-)

Month: 8
Year: 2019

Score = 0.03333333507180214

2 ID: 469
company: Santiane

Comments: Ma de mande et ma conversation avec Sabrina se sont très bien passés aujourd'hui, ce qui n'a pas toujours été le cas avec d'autres conseillers. J'ai attendu 15 jours avant d'avoir une réponse sur un devis de soins dentaires ce qui me parait inaxeptable, surtout quand on vous donne comme réponse ...soyez patient. J'espère qu'a l'avenir tout sera réglé plus rapidement et surtout avec un peu plus d'enpatie.

Month: 6
Year: 2019

Score = 0.032258063554763794

3 ID: 114
company: Réassure

## 5. Sentiment analysis

In [26]:
def sentiment_analysis(prompt, max_retries=3, retry_delay=1):
    """
    ChatGPT with Azure Open AI
    Input: prompt (string)
    Output: Azure Open AI ChatGPT results (string)
    """
    for _ in range(max_retries):
        try:
            response = openai.ChatCompletion.create(
                engine="gpt-35-turbo-16k",  # Should be deployed in the AOAI studio
                messages=[
                    {
                        "role": "system",
                        "content": "You are an AI language model trained to analyze and detect the sentiment \
                        from customers surveys.\
                        Analyze the following comment and determine if the sentiment is: positive, negative or \
                        neutral. Return only a single word, either POSITIVE, NEGATIVE or NEUTRAL. \
                        Just give the answer as the output.\
                        Example: 'Negative' or 'Positive' or 'Neutral'."
                    },
                    {"role": "user", "content": prompt},
                ],
                temperature=0.9,
                max_tokens=200,
                top_p=1,
                frequency_penalty=0,
                presence_penalty=0,
                stop=None,
            )
            resp = response["choices"][0]["message"]["content"]
            
            return resp

        except Exception as e:
            time.sleep(retry_delay)

    return None

In [27]:
sentiment_analysis("I really like this movie")

'Positive'

In [28]:
sentiment_analysis("I never enjoy this")

'Negative'

In [29]:
start = time.time()

query = "soins dentaires"

search_client = SearchClient(acs_endpoint, index_name, AzureKeyCredential(acs_key))

vector = Vector(value=openai_text_embeddings(query), k=10, fields="embed_text")

results = search_client.search(
    search_text=query,
    vectors=[vector],
    filter=myfilter,
    select=["id", "company", "comments", "month", "year"],
    top=10,
)

idx = 1
results_list= []
for result in results:
    print("\033[1;31;34m")
    print("-" * 100)
    print(f"{idx} ID: {result['id']}")
    print("-" * 100)
    print("company:", result["company"])
    print("\033[1;31;32m")
    print(f"Comments: {result['comments']}")
    print("\033[0m")
    print(f"Month: {result['month']}")
    print(f"Year: {result['year']}")
    print("\033[1;31;35m")
    print(f"Score = {result['@search.score']}")
    print("\033[1;31;91m")
    print(">> Sentiment analysis =", sentiment_analysis(result['comments']))
    idx += 1

    results_list.append(result["id"])

elapsed = time.time() - start
print("\033[0m")
print(
    "\nElapsed time: "
    + time.strftime(
        "%H:%M:%S.{}".format(str(elapsed % 1)[2:])[:15], time.gmtime(elapsed)
    )
)


----------------------------------------------------------------------------------------------------
1 ID: 104
----------------------------------------------------------------------------------------------------
company: Réassurez-moi

Comments: Votre cabinet m'a permis de pouvoir bénéficier du contrat de complémentaire santé le plus adéquat en fonction de mes besoins en matière de soins dentaires. Les remboursements des soins se font dans des délais rapides. De plus, je garde le même interlocuteur (merci Charles !) pour toutes les questions que j'ai à poser, et ce dernier est toujours disponible et très aimable :-)

Month: 8
Year: 2019

Score = 0.03333333507180214

>> Sentiment analysis = Positive

----------------------------------------------------------------------------------------------------
2 ID: 469
----------------------------------------------------------------------------------------------------
company: Santiane

Comments: Ma de mande et ma conversation avec Sabrina se so

In [30]:
start = time.time()

query = "soins dentaires avec assurance Néoliane"

search_client = SearchClient(acs_endpoint, index_name, AzureKeyCredential(acs_key))

vector = Vector(value=openai_text_embeddings(query), k=5, fields="embed_text")

results = search_client.search(
    search_text=query,
    vectors=[vector],
    filter=myfilter,
    select=["id", "company", "comments", "month", "year"],
    top=5,
)

idx = 1
results_list= []
for result in results:
    print("-" * 100)
    print(f"{idx} ID: {result['id']}")
    print("-" * 100)
    print("\033[1;31;34m")
    print("company:", result["company"])
    print("\033[1;31;32m")
    print(f"Comments: {result['comments']}")
    print("\033[0m")
    print(f"Month: {result['month']}")
    print(f"Year: {result['year']}")
    print("\033[1;31;35m")
    print(f"Score = {result['@search.score']}")
    print("\033[1;31;34m")
    print("Sentiment analysis of comments:", sentiment_analysis(result['comments']))
    print("\033[0m")
    idx += 1
    results_list.append(result["id"])

elapsed = time.time() - start
print("\033[0m")
print(
    "\nElapsed time: "
    + time.strftime(
        "%H:%M:%S.{}".format(str(elapsed % 1)[2:])[:15], time.gmtime(elapsed)
    )
)

----------------------------------------------------------------------------------------------------
1 ID: 1043
----------------------------------------------------------------------------------------------------

company: Néoliane Santé 

Comments: Bonne Assurance, de ma part je suis très content de la mutuelle Néoliane, des prix raisonnables par rapport aux autres mutuelles ! A recommander

Month: 2
Year: 2019

Score = 0.032522473484277725

Sentiment analysis of comments: Positive

----------------------------------------------------------------------------------------------------
2 ID: 988
----------------------------------------------------------------------------------------------------

company: Néoliane Santé 

Comments: Je suis cliente chez Néoliane depuis 5 ans, aucun soucis à déclarer, remboursements rapides et prestations en adéquation avec le contrat souscris.

Month: 4
Year: 2019

Score = 0.0314980149269104

Sentiment analysis of comments: Positive

-----------------------

## 6. Sentiment analysis with explanations

In [31]:
def sentiment_analysis_with_details(prompt, max_retries=3, retry_delay=1):
    """
    ChatGPT with Azure Open AI
    Input: prompt (string)
    Output: Azure Open AI ChatGPT results (string)
    """
    for _ in range(max_retries):
        try:
            response = openai.ChatCompletion.create(
                engine="gpt-35-turbo-16k",  # Should be deployed in the AOAI studio
                messages=[
                    {
                        "role": "system",
                        "content": "You are an AI language model trained to analyze and detect the sentiment \
                        from customers surveys.\
                        Analyze the following comment and determine if the sentiment is: positive, negative or \
                        neutral. Return only a single word, either POSITIVE, NEGATIVE or NEUTRAL. \
                        Explain."
                    },
                    {"role": "user", "content": prompt},
                ],
                temperature=0.9,
                max_tokens=200,
                top_p=1,
                frequency_penalty=0,
                presence_penalty=0,
                stop=None,
            )
            resp = response["choices"][0]["message"]["content"]
            
            return resp

        except Exception as e:
            time.sleep(retry_delay)

    return None

In [32]:
start = time.time()

query = "soins dentaires avec assurance Néoliane"

search_client = SearchClient(acs_endpoint, index_name, AzureKeyCredential(acs_key))

vector = Vector(value=openai_text_embeddings(query), k=5, fields="embed_text")

results = search_client.search(
    search_text=query,
    vectors=[vector],
    filter=myfilter,
    select=["id", "company", "comments", "month", "year"],
    top=5,
)

idx = 1
results_list= []
for result in results:
    print("-" * 100)
    print(f"{idx} ID: {result['id']}")
    print("-" * 100)
    print("\033[1;31;34m")
    print("company:", result["company"])
    print("\033[1;31;32m")
    print(f"Comments: {result['comments']}")
    print("\033[0m")
    print(f"Month: {result['month']}")
    print(f"Year: {result['year']}")
    print("\033[1;31;35m")
    print(f"Score = {result['@search.score']}")
    print("\033[1;31;34m")
    print("Sentiment analysis of comments:", sentiment_analysis_with_details(result['comments']))
    print("\033[0m")
    idx += 1
    results_list.append(result["id"])

elapsed = time.time() - start
print("\033[0m")
print(
    "\nElapsed time: "
    + time.strftime(
        "%H:%M:%S.{}".format(str(elapsed % 1)[2:])[:15], time.gmtime(elapsed)
    )
)

----------------------------------------------------------------------------------------------------
1 ID: 1043
----------------------------------------------------------------------------------------------------

company: Néoliane Santé 

Comments: Bonne Assurance, de ma part je suis très content de la mutuelle Néoliane, des prix raisonnables par rapport aux autres mutuelles ! A recommander

Month: 2
Year: 2019

Score = 0.032522473484277725

Sentiment analysis of comments: POSITIVE

The sentiment expressed in the comment is positive. The customer states that they are "très content" (very happy) with the Neoliane insurance, highlighting the reasonable prices compared to other insurances. They also mention that they would recommend it. Overall, the comment reflects a positive experience and satisfaction with the insurance.

----------------------------------------------------------------------------------------------------
2 ID: 988
------------------------------------------------------

In [33]:
start = time.time()

query = "problèmes de remboursements"

search_client = SearchClient(acs_endpoint, index_name, AzureKeyCredential(acs_key))

vector = Vector(value=openai_text_embeddings(query), k=5, fields="embed_text")

results = search_client.search(
    search_text=query,
    vectors=[vector],
    filter=myfilter,
    select=["id", "company", "comments", "month", "year"],
    top=5,
)

idx = 1
results_list= []
for result in results:
    print("-" * 100)
    print(f"{idx} ID: {result['id']}")
    print("-" * 100)
    print("\033[1;31;34m")
    print("company:", result["company"])
    print("\033[1;31;32m")
    print(f"Comments: {result['comments']}")
    print("\033[0m")
    print(f"Month: {result['month']}")
    print(f"Year: {result['year']}")
    print("\033[1;31;35m")
    print(f"Score = {result['@search.score']}")
    print("\033[1;31;34m")
    print("Sentiment analysis of comments:", sentiment_analysis_with_details(result['comments']))
    print("\033[0m")
    idx += 1
    results_list.append(result["id"])

elapsed = time.time() - start
print("\033[0m")
print(
    "\nElapsed time: "
    + time.strftime(
        "%H:%M:%S.{}".format(str(elapsed % 1)[2:])[:15], time.gmtime(elapsed)
    )
)

----------------------------------------------------------------------------------------------------
1 ID: 1028
----------------------------------------------------------------------------------------------------

company: Néoliane Santé 

Comments: J'attends toujours les remboursements de l'année 2018. J'ai résilié au 31/12/2018. Ils n'ont pas voulu prétextant que je n'étais pas cliente depuis plus d'un an alors que j'étais cliente depuis plus de 2 ans. Ils ne répondent jamais. Beaucoup de difficultés déjà à la souscription. Envoie de lettres recommandées en octobre,novembre2018 et janvier 2019 demeurées sans aucune réponse Toujours dans l'attente des remboursements.

Month: 2
Year: 2019

Score = 0.030017919838428497

Sentiment analysis of comments: NEGATIVE

The comment expresses frustration and dissatisfaction with the company's handling of the reimbursement process. The customer mentions that they have been waiting for reimbursements from 2018 and that despite being a customer for 